In [ ]:
# We will now extract the feature vectors from an existing network to use them later for clustering our scrapped data. 
# In this notebook the features for the Inception_V3 are extracted exemplarily. 
# However, another Keras model can also be used (https://keras.io/api/applications/).

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
import numpy as np
from tensorflow.keras import preprocessing

In [ ]:
#If you're working with Colab mount your drive or skip this step
from google.colab import drive
drive.mount('/content/drive')

### Building the feature extractor

In [ ]:
from tensorflow.keras.applications import inception_v3

In [ ]:
base_model = inception_v3.InceptionV3(
    include_top = False, # we will not use the predictions
    weights='imagenet'
)

pooling = tf.keras.layers.MaxPooling2D(pool_size=(5, 5))
feature_extractor = tf.keras.Sequential()
feature_extractor.add(base_model)
feature_extractor.add(pooling)

### Building the data pipeline

In [ ]:
scraped_images_folder = '/set/the/path/to/your/scraped/images/'

In [ ]:
filelist_ds = tf.data.Dataset.list_files(scraped_images_folder+'/*')
# If you get an input/output error in Colab, try a few times until the whole dataset is loaded

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224

def load_img(file_path):
    img = tf.io.read_file(file_path)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    # and preprocess
    img = inception_v3.preprocess_input(img)
    return file_path, img

In [ ]:
dataset = filelist_ds.map(load_img)

### Extracting Features

In [ ]:
batched_ds = dataset.batch(64)

feature_collection = {}
for filenames, img in batched_ds:
    features = feature_extractor(img)
    feature_dict = {
        str(name).split('/')[-1]: feature.squeeze() 
        for name, feature in zip(filenames.numpy(), features.numpy())
    }
    feature_collection.update(feature_dict)
    print('*', end='')
features = np.vstack(features).squeeze()

In [ ]:
len(feature_collection)

### Save the data

In [ ]:
feature_file = '/create/a/path/to/save/file/inception_v3_feature.npz'

In [ ]:
np.savez(feature_file, **feature_collection)

In [ ]:
features.shape # Features with shape [batch_size, 2048].